### Introduction

The purpose of this notebook is to illustrate the necessary workflow and programming to process, reconstruct, and finally enhance golden ratio sampled CT scans.
Nothing in this notebook wil run. Instead, the important steps, scripts, classes, and functions are displayed.

Unfortunately, it would not be very readable to go more in-depth than what is here currently presented. The underlying mechanisms can be found in the respective folders.

### Process Golden Ratio Sampled Projections

The first step is to process the obtained projections. 

Assuming several CT scans have been obtained during the same day, it might be beneficial to apply parallelisation.

DynamicProjectionsEQNR is a defined class for reading all folders with projections, performing corrections, finding AoR, rotating the projections, and
initiating a geometry object that can read the scanning settings.

The processed projections are stored to .h5-format using h5py.


In [ ]:
from preprocessing import *
from multiprocessing import Pool

import os, sys
import time
import tqdm


def func(root, expname, oroot, num_proj, nrevs, correction, geometry, roi):
    preprocess = DynamicProjectionsEQNR(
        root,
        expname,
        oroot,
        num_proj,
        nrevs=nrevs,
        correction_parent=correction,
        geometry=geometry,
        roi=roi,
    )

    preprocess()


def main():
    main_root = r"/media/nfs/qnap/home/rubensd/RSD20230509"
    names = [
        "RSD20230509_hourglassV3_13proj",
        "RSD20230509_samplingV3_sandstone_6favg",
        "RSD20230509_sandstoneV3_12favg",
        "RSD20230509_sandstoneV3_24favg",
    ]
    roots = [os.path.join(main_root, name) for name in names]
    roots.append(r"/media/nfs/qnap/home/rubensd/RSD20230428_standard_sandstone")
    main_oroot = r"/media/disks/disk2/CT-data/rubensd/Processed_projections/"
    expnames = [
        "hourglassV3_13_55",
        "limestoneV3_17_55_6favg",
        "limestoneV3_17_55_12favg",
        "limestoneV3_17_55_24favg",
        "limestoneV3_1440_1_std",
    ]
    correction = r"/media/nfs/qnap/home/rubensd/20230424RSD/RSD20230424_samplingV3_hourglass/Corrections"
    num_proj = [13, 17, 17, 17, 1440]
    nrevs = [55, 55, 55, 55, 1]
    geoms = ["golden_motion.nsiprg"] * 4
    geoms.append(None)
    roi = [1536, 1024]

    args = [
        [
            roots[i],
            expnames[i],
            main_oroot,
            num_proj[i],
            nrevs[i],
            correction,
            geoms[i],
            roi,
        ]
        for i in range(len(roots))
    ]

    with Pool() as pool:
        pool.starmap(func, args)


if __name__ == "__main__":
    main()

### Reconstructing the processed projections

With the processed obtained, binning might be necessary. *bin_processed_projections.py* exploits Pytorch average pooling to reduce the spatial size.

The parser arguments are here included. 

---

A python file that has the necessary steps to perform custom golden ratio reconstructions, is *reconstruct_dynamic_main.py*.

Here, different files, time steps, with different number of revolutions, can be reconstructed. 

The core for dealing with golden ratio reconstructions is the *class EquinorDynamicCT(EquinorDataCT):* instance. 

It interacts with the processed projections .h5-file, stores performed reconstructions in a separate .h5-file.

These reconstructions can consist of single revolutions, a custom number of revolutions, and a series (4D).



In [ ]:
import argparse

parser = argparse.ArgumentParser(
    description="Binning of processed projections for CT reconstruction and GAN enhancement"
)

parser.add_argument("-root", type=str, required=True, help="root path")
parser.add_argument(
    "-oroot", type=str, default="", required=False, help="output root path"
)
parser.add_argument("-expname", type=str, required=True, help="Experiment name")
parser.add_argument("-copyname", type=str, required=True, help="Copy name")
parser.add_argument("-bin", type=int, default=2, required=True, help="Binning factor")

! python CT/bin_processed_projections.py -root /media/nfs/qnap/home/rubensd/RSD20230509/RSD20230509_hourglassV3_13proj -expname hourglassV3_13_55 -copyname hourglassV3_13_55_2favg -bin 2


######################



from data_making import *
import numpy as np
import os
import sys
import time
import tqdm
import matplotlib.pyplot as plt


def main():
    root = r"/media/disks/disk2/CT-data/rubensd/Processed_projections/"
    expnames = [
        # "hourglassV3_13_55",
        "limestoneV3_17_55_6favg",
        "limestoneV3_17_55_12favg",
        "limestoneV3_17_55_24favg",
        # "limestoneV3_1440_1_std",
    ]
    oroot = r"/home/rubensd/Documents/DeepLearning/ReconstructionData"

    fibonaccis = [3, 3, 3, 55, 1]
    name = "Fibonacci3"

    for i in tqdm.trange(len(expnames)):
        expname = expnames[i]
        print("Reconstructing", expname)
        model = EquinorDynamicCT(root, expname, oroot, expname)

        if fibonaccis[i] == 1:
            rec = model.reconstruct_idx(idx=0, CoR=0)

            model.save_custom(
                rec, name=name, idx=0, fibonacci=fibonaccis[i], method="fdk"
            )
        else:
            model.reconstruct_custom(idx=0, fibonacci=fibonaccis[i], name=name, CoR=0)


if __name__ == "__main__":
    main()

### Creating dataset for training

The next step in the process is training the GAN in order to perform enhancement of undersampled reconstructions. 

For this, a training dataset is required. 

Similar classes to the one listed above were created to perform simulated data creation, and experimental data creation, respectively.

*process_data* is a parent function for reconstructing all reconstruction instances in a list, and storing the HQ and LQ reconstructions in respective groups with an index equal to the length of the existing dataset.

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt

import data_making as dm
%reload_ext autoreload
%autoreload 2

root = r"F:\RawReconstructions"
exp_names = ["osmoseflom_torrskan kjerne D 05b", "Angola_OC9_S19", "Berea04b", "TR_outcrop_sample", "Vuggy"]
phan_names = [f"phantom_{str(i).zfill(5)}" for i in range(1,12)]
o_root = r"F:\ReconstructionDatasets"
o_name = "experimental_data_factor12"

recs = [dm.EquinorReconstructions(root, name, o_root, o_name) for name in exp_names]
recs[0].process_data(recs, n_angles=12,undersampling_factor=True )

### GAN training

For this, a hyperparameter json file is necessary, as most of the settings are determined by the dictionary stored in this file. 

*create_hparam.py* creates such a json-file. 

Training is initiated in the *gym.py*-file. For the most part, the number of epochs, the paths to different inputs etc. need to be provided. 


In [ ]:
import numpy as np
import pickle as pkl
import torch
import json
import os

hparams = {
    "name": "WGAN",
    "lmse": 1,
    "ladv": 10,
    "lperc": 0,
    "psz": 80,
    "mbsz": 4,
    "itg": 10,
    "itd": 10,
    "lrateg": 1e-5,
    "lrated": 2e-5,
    "train_split": 0.88,
    "transforms": "basic",
    "transfer_model": "simV1_it00500_gen",
}

if __name__ == "__main__":
    my_path = os.path.abspath(os.path.dirname(__file__))

    with open(os.path.join(my_path, "hparams", f"{hparams['name']}.json"), "w+") as d:
        json.dump(hparams, d)

    with open(os.path.join(my_path, "hparams", f"{hparams['name']}.json"), "r") as d:
        dic = json.load(d)

    print(dic)



! python gym.py ...

### GAN inference

With training completed, outputted model weights need to be stored in a safe place. 

The undersampled reconstructions may now be enhanced using the optimised generator. 

*inference.py* and other python files with similar names can perform this task. 

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="3DTomoGAN, load trained model and enhance reconstruction"
    )

    parser.add_argument(
        "-modelPath", type=str, required=True, help="path to model folder"
    )
    parser.add_argument("-modelName", type=str, required=True, help="name of model")
    parser.add_argument("-dataFolder", type=str, required=True, help="Rec folder")
    parser.add_argument("-dataName", type=str, required=True, help="Rec name")
    parser.add_argument("-keyInput", type=str, required=True, help="keyInput")
    parser.add_argument(
        "-keyTarget", type=str, required=False, default="gt", help="keyTarget"
    )
    parser.add_argument(
        "-focus", type=int, required=False, nargs="+", default=0, help="Centre RoI"
    )
    parser.add_argument(
        "-dims", type=int, required=False, nargs="+", default=0, help="Dimensions"
    )

    args, unparsed = parser.parse_known_args()

    # Run Enhancement

    enhance(
        args.modelPath,
        args.modelName,
        args.dataFolder,
        args.dataName,
        args.keyInput,
        key_target=args.keyTarget,
        focus=args.focus,
        dims=args.dims,
    )

### Data Analysis

Performed in Jupyter Notebooks. 

Utilising quality metrics defined in the *utils.py*-file. 

### Final remarks

These are, very shortly summarised, the main steps of the derived technique for GAN-enhanced Golden ratio sampled temporal CT. 

Additional depth can be found in the source code, but readability has not been prioritised; at least not at this current time. 